# Third-Order Letter Approximation Model Tasks
### By Luke Corcoran
### G00410404

In [1]:
# Define necessary imports

import os # For file operations
import collections # For counting characters in files and creating a dictionary
import random # For selecting random items from lists
import json # For exporting model as a JSON file


# Task 1: Third-Order Letter Approximation Model
### Step 1: Initialise Variables and Prepare to Process Files
First, the variables `directory` and `keep` are initialised to store the text file directory and the characters to retain from the text files. The [Counter() function](https://github.com/ianmcloughlin/2425_emerging_technologies/blob/main/02_language_models.ipynb) from Python's 'collections' module is used to keep track of character counts across all files, and a list `cleanedFiles` stores the cleaned content of each file. 

In [2]:
# Directory containing the text files
directory = r'data\utf8_english_works'

# The characters to keep (ASCII, full stops, spaces).
keep = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ .'

# Initialise a Counter to store the frequency of each character across all files
totalCounts = collections.Counter()
    
# Initialise a list to store cleaned files
cleanedFiles = []

### Step 2: Read and Clean Text Files
Next, the [listdir() function in the 'os' module](https://pytutorial.com/python-using-oslistdir-to-list-files-in-a-directory/) is used in a loop to iterate over each file in the folder. It checks if it ends with .txt and joins the file name and the directory path together using another function from the 'os' module, [path.join()](https://www.geeksforgeeks.org/python-os-path-join-method/), in order to access the file via the file path. The content of each text file is read with UTF-8 encoding using the [open()](https://stackoverflow.com/questions/491921/unicode-utf-8-reading-and-writing-to-files-in-python#:~:text=So%20by%20adding%20encoding%3D',of%20everything%20done%20in%20Python.) function. The text is converted to uppercase using the [upper()](https://www.programiz.com/python-programming/methods/string/upper) method. A [generator expression](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#:~:text=in%20Python%20code%3A-,Definition%3A,provided%20within%20the%20parenthetical%20statement.) filters and retains only the characters specified in the `keep` variable, and the [join()](https://discuss.python.org/t/quick-question-on-join/14756) method is used to merge these characters into a string. Finally, using the [append()](https://www.programiz.com/python-programming/methods/list/append) function, the cleaned file is appended to a list of cleaned files for later use.

In [3]:
# Iterate over all files in the directory
for fileName in os.listdir(directory):
    if fileName.endswith('.txt'):
        filePath = os.path.join(directory, fileName)
            
        # Open the file with UTF-8 encoding
        with open(filePath, 'r', encoding='utf-8') as file:
            # Read the whole file into a string.
            english = file.read()

            # Change everything to upper case.
            english = english.upper()

            # Remove unwanted characters.
            cleaned = ''.join(c for c in english if c in keep)

            # Append the cleaned file to the list of cleaned files
            cleanedFiles.append(cleaned)

### Step 3: Extract Main Content from Files and Initialise Word Counts
In this step, a new loop is made which iterates over each cleaned file. Preamble and postamble is removed using both [find()](https://www.w3schools.com/python/ref_string_find.asp) and Python's [slicing function](https://www.w3schools.com/python/ref_func_slice.asp). If the specified start and end markers are found, it keeps only the text between them. It counts the characters in this cleaned text using the [Counter()](https://github.com/ianmcloughlin/2425_emerging_technologies/blob/main/02_language_models.ipynb) function and adds these counts to a total.

In [4]:
# Count the frequency of each character
for cleaned in cleanedFiles:
    # Remove preamble and postamble.If find returns -1, the substring was not found.
    start = cleaned.find('START OF THE PROJECT GUTENBERG EBOOK')
    end = cleaned.find('END OF THE PROJECT GUTENBERG EBOOK')

    # If the substrings are found, extract the main content.
    if start != -1 and end != -1:
        cleaned = cleaned[start:end]
    else:
        print("ERROR: Substrings not found in file:", fileName)
        
    # Count the frequency of each character in the current file
    counts = collections.Counter(cleaned)

    # Update the total counts with the counts from the current file
    totalCounts.update(counts)

### Step 4: Create List of Cleaned Files
To demonstrate the program works so far, the frequency of each character is then printed using a loop that iterates over all the characters and their corresponding counts. The [items()](https://www.programiz.com/python-programming/methods/dictionary/items) function is used to retrieve the key-value pairs (characters and their counts) from `totalCounts`. The cleaned files are stored in a list called `cleanedList` so that they can be used to count the number of sequences in the next step.

In [5]:
# Print the results
for char, count in totalCounts.items():
    print(f"'{char}': {count}")

# Store contents of cleaned files in a list in order to count the number of sequences later
cleanedList = cleanedFiles

'S': 136438
'T': 193301
'A': 171189
'R': 122258
' ': 442809
'O': 159367
'F': 46913
'H': 136849
'E': 265142
'P': 36138
'J': 2418
'C': 49677
'G': 43500
'U': 60490
'N': 147529
'B': 33311
'K': 16941
'L': 87108
'Y': 42416
'I': 144493
'M': 55608
'D': 92967
'W': 50034
'.': 23533
'V': 19577
'X': 2706
'Z': 1117
'Q': 2769


### Step 5: Creating Trigram Model
Finally, using the [defaultdict()](https://www.geeksforgeeks.org/defaultdict-in-python/) function, a dictionary is initialised as the data structure to store the results; this is effective because dictionaries in Python use key-value pairs, which is perfect for storing each trigram as a key and its respective appearance count as a value.

A new loop is made to iterate over each file's content and extract the trigrams that appear in each file by slicing the text from a particular index range using the [range()](https://www.w3schools.com/python/ref_func_range.asp)
 function, incrementing the count of each trigram in the dictionary as it goes. The total count for each trigram is arranged from highest to lowest using the [sorted()](https://www.freecodecamp.org/news/sort-dictionary-by-value-in-python/#:~:text=reverse%20with%20a%20value%20of,sorted%20dictionary%20in%20descending%20order.&text=You%20can%20see%20the%20output,That's%20the%20default.) function. The final result is a sorted dictionary that holds the contents of the trigram model.

In [6]:
# Create a dictionary to store the trigram counts 
trigramCounts = collections.defaultdict(int) 

# Iterate over each cleaned file's content
for cleaned in cleanedList:
    # Iterate over the cleaned text to extract trigrams 
    for i in range(len(cleaned)): 
        trigram = cleaned[i:i+3] # Creates trigram by slicing the cleaned text from index i to i+3
        trigramCounts[trigram] += 1 # Increment the count of the trigram in the dictionary

    # Sort the trigram counts from highest to lowest
    trigramModel = sorted(trigramCounts.items(), key=lambda x: x[1], reverse=True)

# Only displays the top 20 trigrams for brevity
print(trigramModel[:20])

[(' TH', 50120), ('THE', 42657), ('HE ', 33535), ('ED ', 19427), ('AND', 19167), ('ND ', 18886), (' AN', 18522), ('ING', 16298), (' OF', 15054), ('NG ', 14348), (' TO', 14170), ('OF ', 13953), ('TO ', 12595), ('ER ', 12564), (' IN', 12352), ('AT ', 12066), ('IS ', 11072), ('IN ', 10547), (' HE', 10223), ('RE ', 9472)]


### Test #1: Check to see if Preamble and Postamble is being Removed from Each File
This test makes sure that the preamble and postamble is being removed from each file.

The content before the preamble and postamble markers are checked using 'if' and 'in' statements - if they are present, the method returns false, and the test for the respective file will fail.

In [7]:
def ambleRemovedTest(cleanedFile):  
    # Check that the content right before the preamble marker is not in the file
    if "Credits:" in cleanedFile:
        print("Test failed: Preamble not removed.")
        return False

    # Check that the content right after the postamble marker is not in the file
    if "Updated editions will replace the previous one—the old editions will be renamed." in cleanedFile:
        print("Test failed: Postamble not removed.")
        return False

    # Triggers if all tests pass
    return True

# Initialise index for file number
index = 1

# Iterate over each cleaned file to test if preamble and postamble has been removed
for cleaned in cleanedList:
    result = ambleRemovedTest(cleaned)
    if result == True:
        print("Test for file", index, "passed - preamble and postamble has been removed.")
    index += 1

Test for file 1 passed - preamble and postamble has been removed.
Test for file 2 passed - preamble and postamble has been removed.
Test for file 3 passed - preamble and postamble has been removed.
Test for file 4 passed - preamble and postamble has been removed.
Test for file 5 passed - preamble and postamble has been removed.


### Test #2: Check to see if Valid Characters are Present
This test makes sure that each file only contains valid ASCII characters.

Each file is checked for the presence of non-ASCII characters using 'if' and 'not in' statements, looping over each character in the cleaned file. If any are present, the method will return false, and the test for the respective file will fail.

In [8]:
def charValidityTest(cleanedFile):  
    # Define the allowed characters
    asciiCharacters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ ."

    # Check each character to ensure it's in the allowed set
    for char in cleanedFile:
        if char not in asciiCharacters:
            print(f"Test failed: Disallowed character '{char}' found in cleaned content.")
            return False

    # Triggers if all tests pass
    return True

# Initialise index for file number
index = 1

# Iterate over each cleaned file to test if all characters are valid ASCII characters
for cleaned in cleanedList:
    result = charValidityTest(cleaned)
    if result == True:
        print("Test for file", index, "passed - all characters are valid ASCII characters.")
    index += 1

Test for file 1 passed - all characters are valid ASCII characters.
Test for file 2 passed - all characters are valid ASCII characters.
Test for file 3 passed - all characters are valid ASCII characters.
Test for file 4 passed - all characters are valid ASCII characters.
Test for file 5 passed - all characters are valid ASCII characters.


### Test #3: Check to see if All Letters in Each File are Uppercase
This test makes sure that every letter in each file is uppercase.


The check for if all letters are uppercase is done using the [isupper()](https://www.geeksforgeeks.org/check-whether-the-given-character-is-in-upper-case-lower-case-or-non-alphabetic-character/) method, which returns true if all letters are uppercase.

In [9]:
def uppercaseTest(cleanedFile):      
    # Check if all alphabetic characters are uppercase
    if not cleanedFile.isupper():
        print("Test failed: Not all alphabetic characters are uppercase.")
        return False

    # Triggers if all tests pass
    return True

# Initialise index for file number
index = 1

# Iterate over each cleaned file to test if all characters are uppercase
for cleaned in cleanedList:
    result = uppercaseTest(cleaned)
    if result == True:
        print("Test for file", index, "passed - all characters are uppercase.")
    index += 1

Test for file 1 passed - all characters are uppercase.
Test for file 2 passed - all characters are uppercase.
Test for file 3 passed - all characters are uppercase.
Test for file 4 passed - all characters are uppercase.
Test for file 5 passed - all characters are uppercase.


### Test #4: Check for if Trigram Models contain Valid Trigrams
This test checks that each Trigram model only has valid trigrams (with three characters). 

It takes the trigram model as an argument and iterates over each trigram, checking if each one has exactly three characters. For any trigram that does not have exactly three characters, it will return False, printing out the invalid trigram.

In [10]:
def trigramModelTest(trigramModel):     
    # Check each trigram in the model
    for trigram, count in trigramModel:
        # Check if each trigram is exactly 3 characters
        if len(trigram) != 3:
            print(f"Test failed: Trigram '{trigram}' is not 3 characters long.")
            return False
        else:
            return True

# Run the test with this cleaned text
result = trigramModelTest(trigramModel)
if result == True:
    print("Test passed - trigram model contains valid trigrams.")

Test passed - trigram model contains valid trigrams.


# Task 2: Third Order Letter Approximation Generation

### Step 1: Initialise Starting Trigrams and Weights
To begin this task, trigrams beginning with TH are found and weights are created using [list comprehension](https://chatgpt.com/share/670583e8-8ca0-800f-bddd-9e8e27b62db1), with the weights being based off each trigram's reoccurence in the model. Both the trigrams beginning with TH along with their odds of being picked are then displayed using the [zip()](https://www.geeksforgeeks.org/zip-in-python/) function to merge the corresponding lists of trigrams and weights.

In [11]:
# Use list comprehension to find trigrams that start with 'TH'
thKeys = [key for key, value in trigramModel if key.startswith('TH')]

# Create weights based on reoccurrence of trigrams in the model
weights = [value for key, value in trigramModel if key in thKeys]

# Calculate the total weight
totalWeight = sum(weights)

# Print trigrams and weights alongside each other as key-value pairs with total weight as denominator
print("Trigrams: | Odds of being chosen:")
for key, weight in zip(thKeys, weights):
    print(f"{key}       | {weight}/{totalWeight}")

Trigrams: | Odds of being chosen:
THE       | 42657/67788
THA       | 8266/67788
TH        | 5632/67788
THI       | 5589/67788
THO       | 2667/67788
THR       | 1349/67788
THU       | 353/67788
THY       | 279/67788
THS       | 254/67788
TH.       | 214/67788
THT       | 100/67788
THL       | 74/67788
THW       | 65/67788
THF       | 57/67788
THD       | 51/67788
THH       | 50/67788
THM       | 33/67788
THP       | 23/67788
THC       | 18/67788
THN       | 13/67788
THB       | 12/67788
THQ       | 12/67788
THG       | 10/67788
THK       | 5/67788
THJ       | 3/67788
THV       | 2/67788


### Step 2: Initialise Start of 10,000 Character String with Random Trigram
A trigram is then picked at random using the [random.choices()](https://github.com/ianmcloughlin/2425_emerging_technologies/blob/main/02_language_models.ipynb) function, [which bases its selection off weights](https://pynative.com/python-weighted-random-choices-with-probability/); with the more reoccuring trigrams having a higher chance of being selected such as 'THE', 'THA' etc. Using the [str()](https://www.w3schools.com/python/ref_func_str.asp)
 function the chosen trigram is then converted to a string, and the beginning of the 10,000 character string is initialised.

In [12]:
# Pick a trigram based on the weights, using [0] to extract the first element of the list 
chosenTrigram = random.choices(thKeys, weights)[0]

# Convert the randomly chosen trigram to a string
chosenTrigram = str(chosenTrigram)

# Print the beginning of the 10,000 character string
print("Chosen trigram to begin the string:", chosenTrigram)

# Initialise the generated string with the chosen trigram
generatedString = chosenTrigram

Chosen trigram to begin the string: THS


### Step 3: Generate More Characters to Add to the String
Finally, the string needs more characters added to it to reach a length of 10,000 characters. In order to do this, a loop is created that adds characters to the string. Python's [string slicing](https://pythonexamples.org/python-string-get-last-n-characters/) function is used to get the last two characters of the string. [List comprehension](https://chatgpt.com/share/670583e8-8ca0-800f-bddd-9e8e27b62db1) is then used to both find trigrams in the model that begin with these two characters and create weights based off their reoccurence, and the [random.choices()](https://www.w3schools.com/python/ref_random_choices.asp) function is used to randomly select one of the third letters of those trigrams based off their weights. For each third letter that is selected, it is added to the string until it reaches 10,000 characters.

In [13]:
# Loop until the string reaches the desired length (10,000 characters)
while len(generatedString) < 10000:
    # Get the last two characters of the current generated string
    lastTwoChars = generatedString[-2:]
        
    # Use list comprehension to find trigams that start with the last two characters
    possibleTrigrams = [key for key, value in trigramModel if key.startswith(lastTwoChars)]
        
    # Create weights based on reoccurrence of trigrams in the model
    weights = [value for key, value in trigramModel if key in possibleTrigrams]
        
    # Pick a trigram based on the weights
    chosenTrigram = random.choices(possibleTrigrams, weights)[0]
        
    # Add the third character of the chosen trigram to the generated string
    generatedString += chosenTrigram[2]

### Test #1: Check to see that the String is 10,000 Characters Long.
This test verifies that the generated string reaches exactly 10,000 characters.

If successful, it outputs a success message along with the first 100 characters for quick verification. If the string is not 10,000 characters, it prints an error message showing the actual length.


In [14]:
# Ensure the generated string is exactly 10,000 characters long
if len(generatedString) == 10000:
    # Print the first 100 characters for verification
    print("Test Passed: Generated String (first 100 characters):", generatedString[:100])
else:
    # Return the generated string with an error message
    print("Test Failed: The generated string is not 10,000 characters, it is only", len(generatedString), "characters long.")

Test Passed: Generated String (first 100 characters): THS OF WOR STIM THAREELIAT DAME DOWELFAT BAW HIPWAS SHE OF HE OF THERLESS THE BY OF TOICHAL BE SUP A


### Test #2: Check for Valid Characters in Generated String
Checks to ensure that the generated string contains only allowed characters (ASCII characters).

Creates two sets comprised of both the characters in the generated string and characters that are allowed in the string, finding which characters are invalid by [subtracting the valid character set from the string character set](https://docs.python.org/3/library/stdtypes.html#set). Any characters in the difference are stored in `disallowedChars`, and are invalid characters; they will cause the test to fail if present.

In [15]:
# Create sets with characters in the generated string and valid (ASCII) characters
asciiChars = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ .")
charsInString = set(generatedString)

# Find any disallowed characters by using set difference
disallowedChars = charsInString - asciiChars

if not disallowedChars:
    print("Test Passed: All characters in the string are valid.")
else:
    print("Test Failed: Found disallowed character(s):", disallowedChars)

Test Passed: All characters in the string are valid.


# Task 3: Analyze Your Model

### Step 1: Prepare English Words List and Split the Generated String
To begin this task, the path to a file called words.txt, which contains every word in the English dictionary, is stored in the variable `wordsFile`. The [split()](https://stackoverflow.com/questions/6181763/converting-a-string-to-a-list-of-words) function is used to split the `words.txt` file into a list of individual words, and [set()](https://chatgpt.com/share/671c3805-17b0-800f-945a-29c947e9b23e) is used to remove duplicates, creating a complete set of English words. In order to split the 10,000 character string into substrings that could possibly be words for future comparison, the `split()` function is used once more, with the result being stored in `wordsInString`. 


In [16]:
# Assign the path to the words file to a variable 
wordsFile = r'data\english_words\words.txt'

# Read the list of English words from the file, creating a set of english words
with open(wordsFile, 'r') as file:
    englishWords = set(file.read().split())

# Split the extended string into individual substrings
wordsInString = generatedString.split()

### Step 2: Calculate the Percentage of English Words in the String
To calculate the percentage of English words, a [generator expression](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#:~:text=in%20Python%20code%3A-,Definition%3A,provided%20within%20the%20parenthetical%20statement.) is used to iterate over the contents of both the English words list and the generated string, allowing us to count the number of actual English words in the generated string. The percentage of valid English words relative to the total number of words in the string is then calculated using basic math and the result is printed.

In [17]:
# Count the number of English words. The generator yields 1 for every word found in the set of English words.
validWordCount = sum(1 for word in wordsInString if word in englishWords)

# Calculate the percentage of valid English words
totalWords = len(wordsInString)
percentage = (validWordCount / totalWords) * 100

# Print the percentage of valid English words
print(f"Percentage of valid English words: {percentage:.2f}%")

Percentage of valid English words: 34.44%


### Test #1: Ensure English Word Set from words.txt Contains No Non-Alphabetic Entries
This test checks that the set of English words loaded from words.txt contains only alphabetic words.

[List comprehension](https://realpython.com/list-comprehension-python/#using-conditions-in-list-comprehensions) is used to check if the words in `englishWords` are non alphabetical, using the [isalpha()](https://www.programiz.com/python-programming/methods/string/isalpha) function for comparison. If any non-alphabetical words are present in `nonAlphabeticalWords`, the test will fail.

In [18]:
# List comprehension to find non-alphabetic words in the set of English words
nonAlphabeticWords = [word for word in englishWords if not word.isalpha() and word == "-" and word == "'"]

# Check if the list of non-alphabetic words is empty
if not nonAlphabeticWords:
    print("Test Passed: English words set contains alphabetic entries")
else:
    print("Test Failed: English words set contains non-alphabetic entries:", nonAlphabeticWords)

Test Passed: English words set contains alphabetic entries


### Test #2: Test for Presence of Common English Words in words.txt
Checks that a list of common English words (such as "THE", "OF", "AND" etc.) are present in the englishWords set, verifying the validity of the file.

[List comprehension](https://realpython.com/list-comprehension-python/#using-conditions-in-list-comprehensions) is used to filter the words in `commonWords` that are found in `englishWords`. The test compares the length of the resulting list, `commonWordsFound`, with the length of `commonWords`. If all words from commonWords are found in englishWords, the test passes. Otherwise, it prints the missing words by [finding the difference in both sets](https://docs.python.org/3/library/stdtypes.html#set).

In [19]:
# List of common English words
commonWords = ["THE", "OF", "AND", "TO", "IN", "THAT", "IS", "IT", "FOR", "ON", 
                        "WITH", "AS", "AT", "BY", "THIS"]

# Check presence of common words in words.txt
commonWordsFound = [word for word in commonWords if word in englishWords]

# Test passes if all common words are found in the set of English words
if len(commonWordsFound) == len(commonWords):
    print("Test Passed: All words in the common words list are present in the set of English words.")
else:
    print("Test Failed: Words missing from the common words list:", set(commonWords) - set(commonWordsFound))


Test Passed: All words in the common words list are present in the set of English words.


# Task 4: Exporting Model as JSON
### Step 1: Creating Method to Convert Model to JSON File
A simple method named `convertModelToJSON` is used to create a JSON file using the [json.dump()](https://www.geeksforgeeks.org/convert-python-list-to-json/) function, which is ideal for converting lists to JSON files. In this case, it will be used to convert the trigram model, which is in list format, into the required JSON file.

In [20]:
def convertModelToJSON(trigramModel):
    # Export the trigram model as a JSON file
    with open('trigrams.json', 'w') as file:
        json.dump(trigramModel, file)

### Step 2: Export the trigram model as a JSON file
The trigram model which was created in Task 1 is passed in as an argument to `convertModelToJSON`, exporting the model as a JSON file.

In [21]:
# Convert the trigram model to a JSON file
convertModelToJSON(trigramModel)

## Summary

This notebook aims to provide a comprehensive implementation of a third-order letter approximation model. By running through each task step-by-step, it demonstrates the process of cleaning and preparing text data, generating a trigram model to capture character sequences, and using this model to create a string of text based on probability. The final part of the program compares this string of text with words from the English dictionary to determine what percentage of the generated string is valid English words. Finally, the trigram model is exported as a JSON file. This project highlights key computational linguistics techniques and shows the importance of cleaning data, designing algorithms, and carefully evaluating results to build reliable language models.